# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [5]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [6]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("../../data/proofs/english/multiple_solutions/sum_of_two_even_ints/proof1.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()

# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)
informal_proof = re.sub(r"\\subsection\{([^}]+)\}", r"### \1", informal_proof)
informal_proof = re.sub(r"\\title\{([^}]+)\}", r"# \1", informal_proof)
informal_proof = re.sub(r"\\maketitle", "", informal_proof)

# Replace \begin{itemize} and \end{itemize} with Markdown-style lists
informal_proof = re.sub(r"\\begin{itemize}", "", informal_proof)
informal_proof = re.sub(r"\\end{itemize}", "", informal_proof)
informal_proof = re.sub(r"\\item\s+\*\*([^:]+):\*\*", r"- **\1:**", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [7]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Triplet(entities=[Entity(id='1', name='a', label='even integer', type='variable'), Entity(id='2', name='b', label='even integer', type='variable'), Entity(id='3', name='m', label='integer', type='variable'), Entity(id='4', name='n', label='integer', type='variable'), Entity(id='5', name='a + b', label='sum of two even integers', type='expression'), Entity(id='6', name='2(m + n)', label='factored sum', type='expression'), Entity(id='7', name='even integer', label='conclusion', type='definition')], relations=[Relation(source='1', target='3', type='defines', name='definition of even integer'), Relation(source='2', target='4', type='defines', name='definition of even integer'), Relation(source='1', target='5', type='contributes to', name='sum of integers'), Relation(source='2', target='5', type='contributes to', name='sum of integers'), Relation(source='5', target='6', type='factored into', name='factoring out 2'), Relation(source='6', target='7', type='concludes', name='sum is even')])

## 3. Store Triplets into Neo4J DB

In [ ]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("fanavaran", "fanavaran"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()